# Reading and cleaning bike trip history data

This notebook includes initial exploration of the NOAA weather data. Specifically, it was used to:
1. Develop code for cleaning NOAA data prior to merge with tripdata.

In [36]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv('./../data/external/903571.csv', na_values=-9999, parse_dates=[2])

In [38]:
df.head()

,STATION,STATION_NAME,DATE,PRCP,SNWD,SNOW,TAVG,TMAX,TMIN,AWND,WSF2,WSF5,WT01,WT06,WT02,WT04,WT08
0,GHCND:USW00094728,NY CITY CENTRAL PARK NY US,2016-01-01,0.0,0.0,0.0,NaN,42,34,7.6,17.0,25.9,NaN,NaN,NaN,NaN,NaN
1,GHCND:USW00094728,NY CITY CENTRAL PARK NY US,2016-01-02,0.0,0.0,0.0,NaN,40,32,6.7,15.0,23.0,NaN,NaN,NaN,NaN,NaN
2,GHCND:USW00094728,NY CITY CENTRAL PARK NY US,2016-01-03,0.0,0.0,0.0,NaN,45,35,7.8,17.0,25.9,NaN,NaN,NaN,NaN,NaN
3,GHCND:USW00094728,NY CITY CENTRAL PARK NY US,2016-01-04,0.0,0.0,0.0,NaN,36,14,8.5,21.0,31.1,NaN,NaN,NaN,NaN,NaN
4,GHCND:USW00094728,NY CITY CENTRAL PARK NY US,2016-01-05,0.0,0.0,0.0,NaN,29,11,5.1,19.9,30.0,NaN,NaN,NaN,NaN,NaN


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 17 columns):
STATION         366 non-null object
STATION_NAME    366 non-null object
DATE            366 non-null datetime64[ns]
PRCP            366 non-null float64
SNWD            366 non-null float64
SNOW            366 non-null float64
TAVG            0 non-null float64
TMAX            366 non-null int64
TMIN            366 non-null int64
AWND            356 non-null float64
WSF2            358 non-null float64
WSF5            352 non-null float64
WT01            131 non-null float64
WT06            2 non-null float64
WT02            6 non-null float64
WT04            1 non-null float64
WT08            99 non-null float64
dtypes: datetime64[ns](1), float64(12), int64(2), object(2)
memory usage: 48.7+ KB


Based on the number of missing values (and rarity of weather events WT02, WT04, and WT06 which are heavy fog, ice pellets/sleet, and glaze or rime), we will only use the following features:
    - PRCP: Precipitation
    - SNOW: Snowfall
    - SNWD: Snow depth
    - TMAX: Max temperature
    - TMIN: Min temperature
    - AWND: Average daily wind speed
    - WSF2: Fastest 2-minute wind speed
    - WSF5: Fastest 4-second wind speed
    - WT01: Fog, ice fog, or freezing fog (may include heavy fog)
    - WT08: Smoke or haze

In [40]:
df = df[['DATE','PRCP','SNOW','SNWD','TMAX','TMIN','AWND','WSF2','WSF5', 'WT01', 'WT08']]

In [41]:
df = df.set_index('DATE')

In [42]:
df.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,AWND,WSF2,WSF5,WT01,WT08
DATE,,,,,,,,,,
2016-01-01,0.0,0.0,0.0,42,34,7.6,17.0,25.9,NaN,NaN
2016-01-02,0.0,0.0,0.0,40,32,6.7,15.0,23.0,NaN,NaN
2016-01-03,0.0,0.0,0.0,45,35,7.8,17.0,25.9,NaN,NaN
2016-01-04,0.0,0.0,0.0,36,14,8.5,21.0,31.1,NaN,NaN
2016-01-05,0.0,0.0,0.0,29,11,5.1,19.9,30.0,NaN,NaN


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366 entries, 2016-01-01 to 2016-12-31
Data columns (total 10 columns):
PRCP    366 non-null float64
SNOW    366 non-null float64
SNWD    366 non-null float64
TMAX    366 non-null int64
TMIN    366 non-null int64
AWND    356 non-null float64
WSF2    358 non-null float64
WSF5    352 non-null float64
WT01    131 non-null float64
WT08    99 non-null float64
dtypes: float64(8), int64(2)
memory usage: 31.5 KB


Next,we fill the null weather type values with 0's.

In [44]:
df = df.fillna({'WT01':0, 'WT08':0})

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366 entries, 2016-01-01 to 2016-12-31
Data columns (total 10 columns):
PRCP    366 non-null float64
SNOW    366 non-null float64
SNWD    366 non-null float64
TMAX    366 non-null int64
TMIN    366 non-null int64
AWND    356 non-null float64
WSF2    358 non-null float64
WSF5    352 non-null float64
WT01    366 non-null float64
WT08    366 non-null float64
dtypes: float64(8), int64(2)
memory usage: 31.5 KB


To finish cleaning the weather data, all we need to do is fill missing wind speed values. We'll do so by averaging the forward and backfill values (under the intuition that wind speed is essentially continuous, so the mean value is a reasonable estimate).

In [46]:
def get_forward_back_avg(series):
    forward = series.ffill()
    back = series.bfill()
    if np.sum(forward - back) == 0:
        print 'No change for {}'.format(series.name)
    average = (forward + back)/2.0
    return average

In [47]:
for col in df.columns:
    df[col] = get_forward_back_avg(df[col])

No change for PRCP
No change for SNOW
No change for SNWD
No change for TMAX
No change for TMIN
No change for WT01
No change for WT08
